<a href="https://colab.research.google.com/github/jchanlau/NLP_colab/blob/main/nlp_pilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pilot program to distill text from PDF and perform sentiment analysis**

In [ ]:
!pip install transformers        # Hugging Face transfomers
!pip install --upgrade pymupdf   # util to manipulate PDF files
!pip install nltk
# !pip install PyPDF2            # alternative 

from transformers import pipeline # import Hugging Face
import fitz # pymupdf
import nltk # parse sentences in paragraph
nltk.download('punkt')
import numpy as np; import pandas as pd  # standard libraries
import os   # in Google Colab, the home directory is content, 
            # need to store data files in content/data
import re


In [2]:
# Useful functions

# find text between first and last value, avoids repetitions
def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

# find text between first and last value
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

# Parse section into sentences
def parse_sentences(text, classifier, min_chars = 50):
  sent = nltk.sent_tokenize(text)
  sentences = []
  sentiment = []
  for s in sent:
    if len(s)> min_chars:
      sentiment.append(pd.DataFrame(classifier(s)))
      sentences.append(s) 
  sentiment = pd.concat(sentiment).reset_index()
  return sentences, sentiment

# check if sentences match sentiment
def examine_sentiment(sentiment_list, sentence_list, sentiment):
  idx = sentiment_list.index[sentiment_list.label==sentiment].tolist()
  return [sentence_list[i] for i in idx]

# Parse section in paragraphs
def parse_paragraph(text, classifier, min_chars = 20):
  t = text.replace('\n', ' ')
  t = re.split(r'\s\d{1,2}\.\s', t)
  paragraphs = []
  sentiment = []
  for s in t:
    if len(s) > min_chars:
      sentiment.append(pd.DataFrame(classifier(s)))
      paragraphs.append(s)   
  sentiment = pd.concat(sentiment).reset_index()
  return paragraphs, sentiment


In [3]:
# get text
doc = fitz.open("data/1PEREA2021001.pdf")
text= ""
for page in doc:
    text += page.get_text()
doc.close()

context = find_between_r(text, "CONTEXT", "RECENT DEVELOPMENTS")
#print(context)
#context_sentences = nltk.sent_tokenize(context_section)
#context_sentences = parse_sentences(context)


In [ ]:
classifier = pipeline("sentiment-analysis")

sentence_list, sentence_sa = parse_sentences(context, classifier)
paragraph_list, paragraph_sa= parse_paragraph(context, classifier)
print(sentence_sa)

examine_sentiment(sentence_sa, sentence_list, 'NEGATIVE')



